<a href="https://colab.research.google.com/github/RAGHAVA1311/BATCH_05_AIML/blob/main/Copy_of_paddytrail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.utils import image_dataset_from_directory


In [ ]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import os
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.utils import image_dataset_from_directory


In [ ]:
!pip install -q kaggle


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"raghavgaming","key":"e93453c9d3af47cdd581e796c739d277"}'}

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle dataset list

usage: kaggle [-h] [-v] [-W] {competitions,c,datasets,d,kernels,k,models,m,files,f,config} ...
kaggle: error: argument command: invalid choice: 'dataset' (choose from 'competitions', 'c', 'datasets', 'd', 'kernels', 'k', 'models', 'm', 'files', 'f', 'config')


In [ ]:
!kaggle competitions download -c paddy-disease-classification

 98% 1.00G/1.02G [00:07<00:00, 129MB/s]
100% 1.02G/1.02G [00:07<00:00, 140MB/s]


In [ ]:
!unzip paddy-disease-classification.zip


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import os
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
df = pd.read_csv('/content/train.csv')
df.shape

(10407, 4)

In [ ]:
df.head()

,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45


In [ ]:
plt.figure(figsize=(10,10))
sb.countplot(data=df, x='variety')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sb.countplot(df['label'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sb.countplot(df['age'])
plt.show()

In [ ]:
base_path = '/content/train_images'
diseases = os.listdir(base_path)


for dis in diseases:
    disease_specific = df[df['label']==dis]

    index_list = disease_specific.index
    fig, ax = plt.subplots(1, 3, figsize=(15,5))
    fig.suptitle(f'Images for {dis} category . . . .', fontsize=20)
    for i in range(3):
        index = np.random.randint(0,len(index_list))
        index = index_list[index]
        data = df.iloc[index]


        image_path = f'{base_path}/{dis}/{data[0]}'

        img = np.array(Image.open(image_path))
        ax[i].imshow(img)
        ax[i].set_title(f'Type: {data[2]} Age: {data[3]}', fontsize=13)
        ax[i].axis('off')  #As all the images are of 480 x 640 dimension as per the data section.
    plt.show()

In [ ]:
labels = os.listdir('/content/train_images')
img_size = 400

train_dataset = image_dataset_from_directory(
    '/content/train_images',
    image_size = (img_size,img_size),
    batch_size = 64,
    shuffle=True,
    labels='inferred',
    class_names = labels,
    label_mode = 'categorical',
    color_mode='rgb',
    validation_split = 0.1,
    subset='training',
    seed=22
)


val_dataset = image_dataset_from_directory(
    '/content/train_images',
    image_size = (img_size,img_size),
    batch_size = 64,
    shuffle=False,
    labels='inferred',
    class_names = labels,
    label_mode = 'categorical',
    color_mode='rgb',
    validation_split = 0.1,
    subset='validation',
    seed=22
)

Found 10407 files belonging to 10 classes.
Using 9367 files for training.
Found 10407 files belonging to 10 classes.
Using 1040 files for validation.


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
def get_model(base, preprocessor, img_size):
    inputs = tf.keras.Input(shape=(img_size,img_size,3))
    x = RandomFlip('horizontal')(inputs)
    x = preprocessor(x)
    x = base(x)

    x = tfl.Flatten()(x)
    x = tfl.Dense(1024, activation='relu')(x)
    x = tfl.BatchNormalization()(x)
    x = tfl.Dense(512, activation='relu')(x)
    x = tfl.BatchNormalization()(x)
    x = tfl.Dense(128, activation='relu')(x)
    x = tfl.Dropout(0.15)(x)
    x = tfl.BatchNormalization()(x)
    x = tfl.Dense(64, activation='relu')(x)
    x = tfl.Dropout(0.3)(x)
    x = tfl.BatchNormalization()(x)
    outputs = tfl.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    return model

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

pre_trained_model = tf.keras.applications.MobileNetV2(input_shape=(img_size,img_size,3),
                                         include_top=False,
                                         weights='imagenet')

pre_trained_model.trainable = False

In [ ]:
model = get_model(pre_trained_model, preprocess_input, img_size)

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(train_dataset,validation_data=val_dataset,epochs=5,verbose=1)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:,['loss','val_loss']].plot()
history_df.loc[:,['accuracy','val_accuracy']].plot()
plt.show()

In [ ]:
print('Number of layers in the MobileNetV2 Model : ', len(pre_trained_model.layers))


# Here I am training the layers from 120th layer onwards.
for layer in pre_trained_model.layers[120:]:
    layer.trainable=True

In [ ]:
model_tuned = get_model(pre_trained_model, preprocess_input, img_size)

model_tuned.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history_tuned = model_tuned.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=14,
                    verbose=1)

In [ ]:
test_dataset = image_dataset_from_directory(
    '../input/paddy-disease-classification/test_images',
    image_size = (img_size,img_size),
    batch_size = 64,
    shuffle=False,
    labels=None,
    class_names = None,
    label_mode = None,
    color_mode='rgb',
)

In [ ]:
predictions = model_tuned.predict(test_dataset)
predictions.shape

In [ ]:
ss = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')
ss['label'] = np.argmax(predictions, axis=-1)
ss['label'] = ss['label'].replace([0,1,2,3,4,5,6,7,8,9], labels)
ss.to_csv("Submission.csv",index=False)
ss.head()